In [1]:
import matplotlib.pyplot as plt
import kaldi_io
import numpy as np
from glob import glob 
import collections
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)
import re
import os

In [2]:
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.mixture import GaussianMixture
from sklearn.mixture import BayesianGaussianMixture

In [3]:
emo_map={'Angry':0, 'Fear':1,'Happy':2, 'Neutral':3,'Sad':4, 'Surprise':5}

#### STEP 1：extract features from csv files

In [232]:
csv_dir="/home/gyzhang/projects/unsupervised_acoustic_clustring/lc_csv"

In [233]:
feas_list=[]
emo_id_list=[]
for csv_path in glob(csv_dir+'/*.csv'):
    csv_path_base = os.path.basename(csv_path)
    emotion_id = re.split('\-',csv_path_base)[1]
    emo_id_list.append(emo_map[emotion_id])
    with open(csv_path,'rb') as file_id:
        csv_lines=file_id.readlines()
    fea=csv_lines[-1]
    num_fea=re.split("\,",fea.decode('utf-8'))
    feas_list.append(num_fea[1:-1])

#### STEP 2：train K-means

In [271]:
X=np.array(feas_list,dtype=np.float32)
y=np.array(emo_id_list,np.int32)
# pca = PCA(n_components=100)
# X=pca.fit_transform(X)
X_norm=scale(X)

In [273]:
score_list=[]
for i in range(10):
    est=KMeans(init='k-means++', n_clusters=6, n_init=30)
    est.fit(X_norm)
    # STEP 3：evaluations
    score=metrics.adjusted_rand_score(y, est.labels_)
    score_list.append(score)
print(score_list)
print(np.mean(np.array(score_list)))

[0.45179485593270347, 0.45360550898068647, 0.43954718233884132, 0.44692915121695759, 0.45441965916021909, 0.46192933831594796, 0.44800637205433991, 0.45931746803627643, 0.45375227811477742, 0.45655573994743193]
0.45258575541


**STEP 4: Try with GMM**

In [265]:
score_list=[]
for i in range(1):
    estimator = GaussianMixture(n_components=6,
              covariance_type='diag', max_iter=30, random_state=117,n_init=10,reg_covar=1e-6)
    
    estimator.fit(X_norm)
    y_train_pred = estimator.predict(X_norm)
    # STEP 3：evaluations
    score=metrics.adjusted_rand_score(y, y_train_pred)
    score_list.append(score)
print(score_list)
print(np.mean(np.array(score_list)))



[0.064559698897443757]
0.0645596988974


**STEP 4: Try with ivector**

In [ ]:
ivector_scp_path = "/home/gyzhang/projects/kaldi-trunk/egs/voxceleb/v1/exp/ivectors_train/ivector.scp"

In [ ]:
os.chdir("/home/gyzhang/projects/kaldi-trunk/egs/voxceleb/v1/")

In [ ]:
emo_id_map = {'Angry':1,'Happy':2,"Neutral":3,"Sad":4,"Surprise":5,"Fear":6}

In [ ]:
ivectors_list=[]
emo_id_list = []

In [ ]:
for (utt_id, ivector) in kaldi_io.read_mat_scp(ivector_scp_path):
    emotion_id = re.split('\-',utt_id)[1]
    emo_id_list.append(emo_id_map[emotion_id])
    ivectors_list.append(ivector)

In [ ]:
ivectors=np.concatenate(ivectors_list,axis=0)

In [ ]:
emo_id = np.array(emo_id_list)

In [ ]:
emo_id_list=[]
feas_v = np.array(feas_list)
X_tsne_norm=scale(feas_v)

In [ ]:
#----------------------------------------------------------------------
# Scale and visualize the embedding vectors
def plot_embedding(X, y,title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})
#     plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)

In [ ]:
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
X_tsne = tsne.fit_transform(ivectors)

In [ ]:
plot_embedding(X_tsne,emo_id)

In [ ]:
metrics.adjusted_rand_score(emo_id, est.labels_)

In [ ]:
est_emo=np.reshape(est.labels_,(6,-1))

In [ ]:
np.where(est_emo[0] == 5)[0].shape[0]

In [ ]:
for i in range(6):
    print(np.where(est_emo[5] == i)[0].shape[0])

In [ ]:
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
X_tsne = tsne.fit_transform(feas_v)

In [ ]:
X=X_tsne
x_min, x_max = np.min(X, 0), np.max(X, 0)
X = (X - x_min) / (x_max - x_min)

In [ ]:
for i in range(1800):
    text(X[i, 0], X[i, 1], str(emo_id[i]),
                 color=cm.Set1(emo_id[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})